In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import sys
sys.path.append('/Users/andrew/Desktop/sudoku/src/sudoku')

from board import Board
from solutions import Solutions
import utils

In [2]:
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')
softmax = nn.Softmax()

In [23]:
def vector_encode(board_string):
    dim_x, dim_y, board = board_string.split('.')
    max_digit = int(dim_x) * int(dim_y)
    vector = np.zeros((max_digit*len(board),), dtype=np.float64)
    for i in range(len(board)):
        if board[i] != '0':
            vector[i*max_digit + int(board[i]) - 1] = 1
        else:
            vector[i*max_digit:(i+1)*max_digit] = 1/max_digit
    return vector

def get_board_entries(board_string):
    return np.array(list(board_string[4:]), dtype=np.int)

In [24]:
class Linear(nn.Module):
    def __init__(self, max_digit):
        super(Linear, self).__init__()
        self.max_digit = max_digit
        self.linear = nn.Linear(self.max_digit**3, self.max_digit**3)

    def forward(self, X):
        pre_output = self.linear(X)
        predictions = [softmax(pre_output[i*self.max_digit:(i+1)*self.max_digit]) for i in range(self.max_digit**2)]
        return torch.stack(predictions)

In [42]:
max_digit = 4
model = Linear(max_digit)
model.double()

optimizer = optim.LBFGS(model.parameters(), lr=0.8)
X = torch.tensor(vector_encode('2.2.0134432100400010'))
Y = torch.tensor(get_board_entries('2.2.2134432112433412'), dtype=torch.int64) - 1

def closure():
    optimizer.zero_grad()
    prediction = model(X)
    loss = nn.functional.nll_loss(prediction, Y)
    print(loss)
    loss.backward()
    return loss

In [43]:
for i in range(100):
    optimizer.step(closure)

tensor(-0.2535, grad_fn=<NllLossBackward>)
tensor(-0.2580, grad_fn=<NllLossBackward>)
tensor(-0.2877, grad_fn=<NllLossBackward>)
tensor(-0.3215, grad_fn=<NllLossBackward>)
tensor(-0.3593, grad_fn=<NllLossBackward>)
tensor(-0.4009, grad_fn=<NllLossBackward>)
tensor(-0.4454, grad_fn=<NllLossBackward>)
tensor(-0.4919, grad_fn=<NllLossBackward>)
tensor(-0.5388, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0000, grad_fn=<NllLossBackward>)
tensor(-1.0

/Users/andrew/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [51]:
solutions = Solutions('/Users/andrew/Desktop/sudoku/data/solutions5.txt')

In [54]:
puzzles = solutions.get_puzzles_by_hints()

In [58]:
for hints in puzzles:
    print(hints, len(puzzles[hints]))

16 12
15 188
14 716
13 972
12 899
11 824
10 711
9 533
8 339
7 230
6 135
5 58
4 10


In [54]:
a = torch.tensor(np.array([[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]]), dtype=torch.float64)
b = torch.tensor(np.array([0, 2, 4]), dtype=torch.int64)
a = torch.tensor(np.array([[.2, 0, 0, 0, 0]]), dtype=torch.float64, requires_grad=True)
b = torch.tensor(np.array([0]), dtype=torch.int64)
loss = nn.functional.nll_loss(a, b)

print(a)
print(b)
print(loss)
print(a.shape)
print(b.shape)

tensor([[0.2000, 0.0000, 0.0000, 9.0000, 0.0000]], requires_grad=True)
tensor([0])
tensor(-0.2000, grad_fn=<NllLossBackward>)
torch.Size([1, 5])
torch.Size([1])


In [11]:
nn.functional.nll_loss?

In [ ]:
for i in range(15):
    print('STEP: ', i)
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.item())
        loss.backward()
        return loss
    optimizer.step(closure)
    # begin to predict, no need to track gradient here
    with torch.no_grad():
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('test loss:', loss.item())
        y = pred.detach().numpy()
    # draw the result
    plt.figure(figsize=(30,10))
    plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
    plt.xlabel('x', fontsize=20)
    plt.ylabel('y', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    def draw(yi, color):
        plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
        plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
    draw(y[0], 'r')
    draw(y[1], 'g')
    draw(y[2], 'b')
    plt.savefig('predict%d.pdf'%i)
    plt.close()